Boilerplate code below..

In [55]:
import pandas as pd
import numpy as np
from typing import List, Dict
from IPython.display import display, HTML, Markdown

import warnings
warnings.filterwarnings('ignore')


def display_best_and_worse_recommendations(recommendations: pd.DataFrame):
    recommendations.sort_values('Estimated Prediction', ascending=False, inplace=True)

    top_recommendations = recommendations.iloc[:10]
    top_recommendations.columns = ['Prediction (sorted by best)', 'Movie Title']

    worse_recommendations = recommendations.iloc[-10:]
    worse_recommendations.columns = ['Prediction (sorted by worse)', 'Movie Title']

    display(HTML("<h1>Recommendations your user will love</h1>"))
    display(top_recommendations)

    display(HTML("<h1>Recommendations your user will hate</h1>"))
    display(worse_recommendations)
    

def load_movies_dataset() -> pd.DataFrame:
    movie_data_columns = [
    'movie_id', 'title', 'release_date', 'video_release_date', 'url',
    'unknown', 'Action', 'Adventure', 'Animation', "Children's",
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western'
    ]

    movie_data = pd.read_csv(
        'datasets/ml-100k/u.item', 
        sep = '|', 
        encoding = "ISO-8859-1", 
        header = None, 
        names = movie_data_columns,
        index_col = 'movie_id'
    )
    movie_data['release_date'] = pd.to_datetime(movie_data['release_date'])
    return movie_data

def load_ratings() -> pd.DataFrame:
    ratings_data = pd.read_csv(
        'datasets/ml-100k/u.data',
        sep = '\t',
        encoding = "ISO-8859-1",
        header = None,
        names=['user_id', 'movie_id', 'rating', 'timestamp']
    )
    return ratings_data

# A practical guide to Singular Value Decomposition in Python

Recommender systems have become increasingly popular in recent years, and are used by some of the largest websites in the world to predict the likelihood of a user taking an action on an item. In the world of Netflix, this means recommending similar movies to the ones you have seen. In the world of dating, this means suggesting matches similar to people you already showed interest in!

My path to recommenders has been an unusual one: from a Software Engineer to working on matching algorithms at a dating company, with a little background on machine learning. With my knowledge of Python and the use of basic SVD (Singular Value Decomposition) frameworks, I was able to understand SVDs from a practical standpoint of what you can do with them, instead of focusing on the science.

In my talk, you will learn 2 practical ways of generating recommendations using SVDs: matrix factorization and item similarity. We will be learning the high-level components of SVD the "doer way": we will be implementing a simple movie recommendation engine with the help of Jupiter notebooks, the MovieLens database, and the Surprise recommendation package.

## Table of contents

 - Downloading and exploring the MovieLens dataset
 - Training a SVD model using Surprise
 - Using the predict() API inside of Surprise
 - Recommendations via Matrix Factorization: Performing predict() manually
 - recommendations via Product based CF: Finding similarity between vectors

# MovieLens dataset

This dataset contains all the movies and their metadata

`movie_id` 1 is **Toy Story**

<p><img src="https://static1.squarespace.com/static/51cdafc4e4b09eb676a64e68/t/579282fabebafbb6c366252c/1469219594863/" alt="Drawing" style="width: 200px; float: left"/></p>

In [114]:
movie_data = load_movies_dataset()
ratings_data = load_ratings()
ratings_data.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


# Ratings dataset

Contains the **interactions** between users and movies

- User **196** rated movie **242** with a score of **3** 
- User **186** rated movie **302** with a score of **3** 
- User **22** rated movie **377** with a score of **3** 

In [86]:
ratings_data[ratings_data['movie_id'] == 1]['rating'].describe()

count    452.000000
mean       3.878319
std        0.927897
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

On average, people really LOVE toy story! and I don't blame them!

# Running our interactions through Surprise SVD

Let's take the **interactions** between the Users and Movies, and generate **latent features**  

In [97]:
from surprise import SVD, NMF, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

# Surprise has some preset datasets, including ml-100k! but today I'm going to show you how to load your own
# data = Dataset.load_builtin('ml-100k')

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_data[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25)

# asd = data.build_full_trainset()

# Let's train a new Nonnegative SVD
model = SVD(n_factors=100, biased=False)
model.fit(trainset)

# In reality, we should perform a train/test split and check RMSE to see if our model is trained
# but today, for simplicity, I'm skipping this step
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9553


0.9552646230761732

# Inspecting our Product Matrix

Surprise SVD stores the product matrix under the `model.qi` attribute. Let's take a look

In [98]:
pd.DataFrame(model.qi).head(10)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.285818,0.133365,0.117562,0.531781,-0.097626,-0.151435,-0.089706,0.121677,0.000533,0.052421,...,-0.292722,0.074519,0.250477,0.298131,-0.031393,0.265227,0.110920,-0.261314,0.484507,-0.217312
1,0.078828,0.115741,-0.035391,0.320762,-0.472414,0.222558,0.295761,0.015669,-0.117158,0.079879,...,-0.078050,0.227592,0.035191,0.005143,0.338501,0.314905,0.313024,-0.094643,0.073450,0.181732
2,-0.160768,0.069406,0.206091,0.215977,0.036046,0.007599,-0.104122,0.056567,0.053568,-0.230164,...,-0.205914,0.152749,0.017517,0.005258,0.177150,0.144770,-0.173821,-0.177881,0.050665,-0.041787
3,-0.026420,-0.086641,0.118603,0.150126,0.132124,-0.096485,-0.074533,0.115905,0.347283,-0.265251,...,-0.289227,0.189203,0.218566,0.238332,0.173330,0.131126,-0.020112,-0.203606,0.177268,-0.191234
4,-0.041669,-0.043359,0.094971,0.078061,0.237612,0.116392,-0.065597,0.180761,0.094201,-0.011490,...,-0.006076,0.107478,0.332870,0.485690,-0.024210,0.412115,-0.229576,-0.371279,0.199899,-0.272527
5,-0.053930,-0.045940,0.422947,0.104168,0.138567,-0.117674,-0.207757,0.258196,0.088478,-0.237259,...,-0.134647,-0.268064,0.219690,0.248847,0.141277,0.133228,-0.063858,0.076059,0.130332,-0.183578
6,-0.050339,-0.052890,0.182766,0.052496,-0.148270,0.055639,-0.306368,0.134306,0.091626,-0.324649,...,-0.157279,0.246892,0.266700,0.312718,0.246287,0.254372,-0.026646,0.058665,0.075224,-0.116682
7,0.065413,0.107288,0.065675,-0.042226,-0.059311,0.097559,0.038622,-0.039200,0.059897,-0.038857,...,0.019938,-0.004068,-0.069274,0.093819,0.132113,0.257403,0.079338,-0.020743,0.249408,-0.113026
8,0.042767,-0.092076,0.159719,0.177728,-0.065333,0.118134,-0.233625,0.041633,-0.079350,0.074467,...,0.043600,0.196893,0.087005,0.385093,0.224093,0.409940,0.046564,-0.156038,0.257124,0.000594
9,0.109603,0.484504,0.005266,0.057026,0.204965,0.202200,-0.159601,-0.061320,-0.043362,-0.129514,...,-0.269980,0.232671,0.325092,0.300575,-0.091804,0.178234,0.142365,0.247111,0.411928,0.033036


# Exploring the product matrix

The matrix has `n_factors` columns (we chose 10). Every row represents a movie

In [99]:
print(f"The shape of our product matrix is {model.qi.shape}.")
print(f"There are {ratings_data['movie_id'].unique().shape[0]} unique movies movies")

The shape of our product matrix is (1643, 100).
There are 1682 unique movies movies


Around 3% of movies are not present. This is because Surprise removes products (and users) that do not have a minimum number of ratings. 

# Generating predictions with simplicity

Before looking into the latent features of our movies, let's use the API provided by Surprise. More specifically, Surprise provides us 1 API

 - `model.predict` computes the rating prediction for given user and movie
 
Let's look at how we can use this API to generate movies that a given user may like

```python
>>> model.predict('302', '1')
Prediction(uid=302, iid=1, r_ui=None, est=3.5327866666666665, details={'was_impossible': False})
```

NOTE: User ID and Movie ID are **strings**

In [116]:
svd.predict(196, 1)

3.4258653536152694

In [100]:
movie_id_to_title_map: Dict[int, str] = dict(movie_data['title'])
# {1: 'Toy Story (1995)',
#  2: 'GoldenEye (1995)',
#  3: 'Four Rooms (1995)'}

def generate_recommended_movies_for_user(user_id: int) -> pd.DataFrame:
    """Return a DataFrame containing recommendations for the user, and the
    associated score
    """
    results = []
    for movie_id, movie_title in movie_id_to_title_map.items():
        
        # For each movie, calculate score prediction 
        prediction = model.predict(user_id, movie_id)
        results.append((prediction.est, movie_title))
       
    return pd.DataFrame(results, columns=['Estimated Prediction', 'Movie Title'])


# Let's generate some recommendations for a user
recommendations = generate_recommended_movies_for_user(302)
display_best_and_worse_recommendations(recommendations)

,Prediction (sorted by best),Movie Title
1681,3.5298,Scream of Stone (Schrei aus Stein) (1991)
1475,3.5298,Raw Deal (1948)
710,3.5298,"Substance of Fire, The (1996)"
1543,3.5298,It Takes Two (1995)
1522,3.5298,"Good Man in Africa, A (1994)"
1519,3.5298,"Fear, The (1995)"
1515,3.5298,"Wedding Gift, The (1994)"
1497,3.5298,Farmer & Chase (1995)
1493,3.5298,"Mostro, Il (1994)"
1481,3.5298,"Gate of Heavenly Peace, The (1995)"


,Prediction (sorted by worse),Movie Title
1431,1.0,"Mighty, The (1998)"
1426,1.0,Drunks (1995)
1421,1.0,Suture (1993)
1419,1.0,Gilligan's Island: The Movie (1998)
1417,1.0,"Joy Luck Club, The (1993)"
1416,1.0,"Turning, The (1992)"
1413,1.0,Coldblooded (1995)
1411,1.0,Land Before Time III: The Time of the Great Gi...
1407,1.0,Gordy (1995)
1401,1.0,"Ciao, Professore! (1993)"


# Predict, under the hood

So far we have seen how the `predict()` API works in surface. But how does it **really** work inside of surprise. It's, surprisingly, simple! (get the pun?)

But before we go there, let's go back to our Feature Vectors

![Latent Features](https://cdn-images-1.medium.com/max/1600/0*_gKhyxIC3wup0cCE.jpg)

## Looking at the Movie matrix (vT)

Let's take a look at the latent features for every movie. Product features can be found in the `qi` attribute.
 - create a DataFrame that maps product matrix row index to movie
 - join the newly created dataframe with the movie dataset
 - join the newly created dataframe with the latent features

In [101]:
# Create a DataFrame that maps product matrix row index to movie
movie_to_product_matrix = pd.DataFrame(
    list(model.trainset._raw2inner_id_items.items()
), columns=['movie_id', 'vT_index'], dtype=int).set_index('movie_id', drop=False)

# Join the newly created dataframe with the movie dataset
mapping_matrix_with_title = movie_to_product_matrix.join(movie_data['title'])

# Create a dataframe containing latent features, and join it to the remaining dataset
latent_features = pd.DataFrame(model.qi, columns=[f"Latent Feature {k}" for k in range(1, 101)])
mapping_matrix_with_title_and_features = mapping_matrix_with_title.set_index('vT_index').join(latent_features)

mapping_matrix_with_title_and_features.head(10)

,movie_id,title,Latent Feature 1,Latent Feature 2,Latent Feature 3,Latent Feature 4,Latent Feature 5,Latent Feature 6,Latent Feature 7,Latent Feature 8,...,Latent Feature 91,Latent Feature 92,Latent Feature 93,Latent Feature 94,Latent Feature 95,Latent Feature 96,Latent Feature 97,Latent Feature 98,Latent Feature 99,Latent Feature 100
vT_index,,,,,,,,,,,,,,,,,,,,,
0,69,Forrest Gump (1994),0.285818,0.133365,0.117562,0.531781,-0.097626,-0.151435,-0.089706,0.121677,...,-0.292722,0.074519,0.250477,0.298131,-0.031393,0.265227,0.110920,-0.261314,0.484507,-0.217312
1,9,Dead Man Walking (1995),0.078828,0.115741,-0.035391,0.320762,-0.472414,0.222558,0.295761,0.015669,...,-0.078050,0.227592,0.035191,0.005143,0.338501,0.314905,0.313024,-0.094643,0.073450,0.181732
2,1263,Foxfire (1996),-0.160768,0.069406,0.206091,0.215977,0.036046,0.007599,-0.104122,0.056567,...,-0.205914,0.152749,0.017517,0.005258,0.177150,0.144770,-0.173821,-0.177881,0.050665,-0.041787
3,73,Maverick (1994),-0.026420,-0.086641,0.118603,0.150126,0.132124,-0.096485,-0.074533,0.115905,...,-0.289227,0.189203,0.218566,0.238332,0.173330,0.131126,-0.020112,-0.203606,0.177268,-0.191234
4,1041,Forget Paris (1995),-0.041669,-0.043359,0.094971,0.078061,0.237612,0.116392,-0.065597,0.180761,...,-0.006076,0.107478,0.332870,0.485690,-0.024210,0.412115,-0.229576,-0.371279,0.199899,-0.272527
5,546,Broken Arrow (1996),-0.053930,-0.045940,0.422947,0.104168,0.138567,-0.117674,-0.207757,0.258196,...,-0.134647,-0.268064,0.219690,0.248847,0.141277,0.133228,-0.063858,0.076059,0.130332,-0.183578
6,19,Antonia's Line (1995),-0.050339,-0.052890,0.182766,0.052496,-0.148270,0.055639,-0.306368,0.134306,...,-0.157279,0.246892,0.266700,0.312718,0.246287,0.254372,-0.026646,0.058665,0.075224,-0.116682
7,1229,Poison Ivy II (1995),0.065413,0.107288,0.065675,-0.042226,-0.059311,0.097559,0.038622,-0.039200,...,0.019938,-0.004068,-0.069274,0.093819,0.132113,0.257403,0.079338,-0.020743,0.249408,-0.113026
8,1073,Shallow Grave (1994),0.042767,-0.092076,0.159719,0.177728,-0.065333,0.118134,-0.233625,0.041633,...,0.043600,0.196893,0.087005,0.385093,0.224093,0.409940,0.046564,-0.156038,0.257124,0.000594


These are **learned features**. We cannot attribute them to anything specific, but they usually have some real-world correlation

# Find similar movies using Cosine Similarity

Usually, there isn't a straightforward way to pinpoint what a latent feature may be a strong indicator of. Even though we don't know exactly what these features correlate to, we can still compare vectors together. The latent feature at same index of every vector will relate to the same attribute.

To find how similar 2 movies are, all we need to do is compare their vectors

In [102]:
from scipy.spatial.distance import cosine


def compute_similarity(movie_a: str, movie_b: str) -> float:
    try:
        movie_a_vectors: np.array = mapping_matrix_with_title_and_features[
            mapping_matrix_with_title_and_features['title'] == movie_a
        ].iloc[0, 2:].as_matrix()
        movie_b_vectors: np.array = mapping_matrix_with_title_and_features[
            mapping_matrix_with_title_and_features['title'] == movie_b
        ].iloc[0, 2:].as_matrix()
    except IndexError:
        # SVD may sometimes remove users or products that do not contain
        # a minimum number of ratings to/from them. This helps improve the
        # quality of recommendations
        return -1
    
    return 1 - cosine(movie_a_vectors, movie_b_vectors)


# compute_similarity('Evita (1996)', 'Evita (1996)')
compute_similarity('Toy Story (1995)', 'Evita (1996)')
# compute_similarity('They Made Me a Criminal (1939)', 'Toy Story (1995)')

0.44589190533565615

In [103]:
def generate_similar_movies_for_movie(movie_title: str) -> pd.DataFrame:
    all_movies = movie_data[['title']]
    all_movies['similarity'] = all_movies['title'].map(lambda title: compute_similarity(title, movie_title))
    return all_movies

In [104]:
similarity_table = generate_similar_movies_for_movie('Toy Story (1995)')
similarity_table.sort_values('similarity', ascending=False).head(5)

,title,similarity
movie_id,,
1,Toy Story (1995),1.000000
651,Glory (1989),0.771793
204,Back to the Future (1985),0.750685
164,"Abyss, The (1989)",0.748735
479,Vertigo (1958),0.747471


In [105]:
similarity_table = generate_similar_movies_for_movie('Star Wars (1977)')
similarity_table.sort_values('similarity', ascending=False).head(5)

,title,similarity
movie_id,,
50,Star Wars (1977),1.000000
181,Return of the Jedi (1983),0.838185
172,"Empire Strikes Back, The (1980)",0.832663
210,Indiana Jones and the Last Crusade (1989),0.768443
174,Raiders of the Lost Ark (1981),0.763315


In [106]:
similarity_table = generate_similar_movies_for_movie('Monty Python and the Holy Grail (1974)')
similarity_table.sort_values('similarity', ascending=False).head(5)

,title,similarity
movie_id,,
168,Monty Python and the Holy Grail (1974),1.000000
648,"Quiet Man, The (1952)",0.768556
1007,Waiting for Guffman (1996),0.745965
408,"Close Shave, A (1995)",0.738414
12,"Usual Suspects, The (1995)",0.737224




![Wallace and Grommit](https://images-na.ssl-images-amazon.com/images/M/MV5BYjkyM2Y1NzQtYmQ0Zi00MmE5LTgwY2QtNjI3MmE4NzhmNTUwXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_.jpg)

In [79]:
ratings_data.groupby('movie_id').sum().sort_values('rating', ascending=False).join(movie_data)

,user_id,rating,timestamp,title,release_date,video_release_date,url,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
50,274817,2541,514804231448,Star Wars (1977),1977-01-01,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,...,0,0,0,0,0,1,1,0,1,0
100,231486,2111,448437746740,Fargo (1996),1997-02-14,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,...,0,0,0,0,0,0,0,1,0,0
181,240820,2032,447548365780,Return of the Jedi (1983),1997-03-14,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,...,0,0,0,0,0,1,1,0,1,0
258,235005,1936,449993136137,Contact (1997),1997-07-11,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,...,0,0,0,0,0,0,1,0,0,0
174,199104,1786,370882569779,Raiders of the Lost Ark (1981),1981-01-01,NaN,http://us.imdb.com/M/title-exact?Raiders%20of%...,0,1,1,...,0,0,0,0,0,0,0,0,0,0
127,196558,1769,364769941246,"Godfather, The (1972)",1972-01-01,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,...,0,0,0,0,0,0,0,0,0,0
286,225292,1759,425069350834,"English Patient, The (1996)",1996-11-15,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,215609,1753,399028021059,Toy Story (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,176180,1673,344378020675,"Silence of the Lambs, The (1991)",1991-01-01,NaN,http://us.imdb.com/M/title-exact?Silence%20of%...,0,0,0,...,0,0,0,0,0,0,0,1,0,0
